#  Grover's Algorithm: Cracking a 3-bit Password
This notebook demonstrates Grover's Algorithm step-by-step with **word-by-word explanations**.
We aim to find a secret 3-bit password (e.g., `'101'`) using fewer checks than classical brute-force search.

In [ ]:
# Install Qiskit (only if you're running locally)
# !pip install qiskit

##  Step 1: Import Libraries

In [ ]:
# We import the necessary Qiskit and plotting libraries
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt
import numpy as np

## Step 2: Define the Oracle
The oracle flips the sign of the amplitude of the correct password. It acts like a black-box that returns true only for the secret.

In [ ]:
def password_oracle(n_qubits, password):
    '''Creates a quantum oracle that flips the sign of the state |password⟩'''
    qc = QuantumCircuit(n_qubits)
    # Step 1: Flip qubits where password has 0 (to map |password⟩ to |111⟩)
    for i in range(n_qubits):
        if ((password >> i) & 1) == 0:
            qc.x(i)  # Apply X gate to invert bits with 0 in target

    # Step 2: Multi-controlled-Z using H and MCT
    qc.h(n_qubits-1)  # Put last qubit into phase basis
    qc.mct(list(range(n_qubits-1)), n_qubits-1)  # Apply multi-controlled Toffoli (acts like Z)
    qc.h(n_qubits-1)  # Bring back to normal basis

    # Step 3: Undo the X gates to reset
    for i in range(n_qubits):
        if ((password >> i) & 1) == 0:
            qc.x(i)

    oracle_gate = qc.to_gate()
    oracle_gate.name = f'Oracle({password:03b})'  # Label the gate
    return oracle_gate

## Step 3: Grover Diffusion Operator
This step amplifies the correct state by reflecting all amplitudes around the average.

In [ ]:
def grover_diffusion(n_qubits):
    qc = QuantumCircuit(n_qubits)
    # Step 1: Apply Hadamard to go to equal superposition
    qc.h(range(n_qubits))

    # Step 2: Invert about mean (X -> H -> MCT -> H -> X)
    qc.x(range(n_qubits))
    qc.h(n_qubits-1)
    qc.mct(list(range(n_qubits-1)), n_qubits-1)
    qc.h(n_qubits-1)
    qc.x(range(n_qubits))
    qc.h(range(n_qubits))
    return qc.to_gate(label='Diffusion')

##  Step 4: Build Grover Circuit for 3-bit Password

In [ ]:
n_qubits = 3
password = 5  # Let's say the hidden password is '101' (binary of 5)

# Calculate number of Grover iterations: floor(pi/4 * sqrt(N))
N = 2**n_qubits
k = int(np.floor(np.pi/4 * np.sqrt(N)))
print(f'Grover iterations: {k}')

# Create the quantum circuit
qc = QuantumCircuit(n_qubits, n_qubits)

# Step 1: Apply Hadamard to all qubits to create superposition
qc.h(range(n_qubits))  # This puts all 8 states (000 to 111) in equal superposition

# Step 2: Apply Grover iteration (oracle + diffusion)
oracle = password_oracle(n_qubits, password)
diff = grover_diffusion(n_qubits)
for _ in range(k):
    qc.append(oracle, range(n_qubits))  # Apply the oracle
    qc.append(diff, range(n_qubits))    # Amplify the result

# Step 3: Measure the output
qc.measure(range(n_qubits), range(n_qubits))
qc.draw('mpl')

## Step 5: Simulate the Circuit

In [ ]:
backend = Aer.get_backend('qasm_simulator')  # Use simulator backend
shots = 1024
job = execute(qc, backend=backend, shots=shots)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)
plt.show()
print(f'Highest probability state: {max(counts, key=counts.get)} (password: {password:03b})')

## Step 6: Success Probability vs. Number of Iterations

In [ ]:
probs = []
iters = [1,2,3]
for k in iters:
    qc = QuantumCircuit(n_qubits, n_qubits)
    qc.h(range(n_qubits))
    oracle = password_oracle(n_qubits, password)
    diff = grover_diffusion(n_qubits)
    for _ in range(k):
        qc.append(oracle, range(n_qubits))
        qc.append(diff, range(n_qubits))
    qc.measure(range(n_qubits), range(n_qubits))
    job = execute(qc, backend=backend, shots=shots)
    counts = job.result().get_counts()
    success = counts.get(f'{password:03b}', 0)/shots
    probs.append(success)
plt.plot(iters, probs, 'o-')
plt.xlabel('Number of Grover Iterations')
plt.ylabel('Success Probability')
plt.title('Grover Success Probability vs Iterations')
plt.show()

## Conclusion
- Grover's algorithm finds the password in just ~2 iterations vs 8 classically.
- After just 2 iterations, the probability of success is >90%.
- This shows how quantum algorithms can offer real-world speedups for search problems.

 now we can Try changing the password or increasing the number of qubits to see Grover’s scaling!